In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import glob, os

from sdt_dask.dask_tool.sdt_dask import SDTDask

In [6]:
SDTDask?

Init signature: SDTDask(data_plug, client)
Docstring:      <no docstring>
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dask_tool/sdt_dask.py
Type:           type
Subclasses:     

## Local Client example 
 use local csv dataplug and local client

### Instantiate a localfile data plug

In [7]:
from sdt_dask.dataplugs.csv_plug import LocalFiles

In [8]:
LocalFiles?

Init signature: LocalFiles(path_to_files, ext='.csv')
Docstring:     
Dataplug class for retrieving data from some source. It's recommended
that user-created dataplug inherit from this class to ensure compatibility.

The initialization argument for each class will be different depending on
the source. The main requirement is to keep the ``Dataplug.get_data`` method,
and make sure the args and returns as defined here.
File:           /mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/sdt_dask/dataplugs/csv_plug.py
Type:           type
Subclasses:     

In [9]:
path = "../dataplugs/example_data/"
local_file_data_plug = LocalFiles(path_to_files=path)
local_file_keys = [(os.path.basename(fname)[:-4],) for fname in glob.glob(path + "*")]
local_file_keys

[('TAAI01129193',), ('TABC01142170',), ('TABD01110568',)]

In [10]:
local_file_data_plug.get_data(local_file_keys[1])

Loading file TABC01142170...


,ac_power_01,ac_power_02,ac_power_03
ts,,,
2013-12-31 16:00:00,0.0767,0.0928,0.0928
2013-12-31 16:05:00,0.0695,0.0862,0.0819
2013-12-31 16:10:00,0.0626,0.0726,0.0653
2013-12-31 16:15:00,0.0546,0.0639,0.0613
2013-12-31 16:20:00,0.0418,0.0530,0.0516
...,...,...,...
2019-06-20 15:30:00,0.5872,0.5950,0.2445
2019-06-20 15:35:00,0.6454,0.6506,0.2748
2019-06-20 15:40:00,0.6220,0.6313,0.2645


### Set a local User-defined Client

In [11]:
from dask.distributed import Client

In [12]:
n_workers = 1
threads_per_worker = 1
total_system_memory = 8
memory_per_worker = total_system_memory / n_workers
local_client = Client(processes=False, memory_spill_fraction=False, memory_pause_fraction=False, memory_target_fraction=0.8, n_workers=n_workers, threads_per_worker=threads_per_worker, memory_limit=f"{memory_per_worker}GiB")
local_client.dashboard_link

2024-03-23 11:54:43,146 - distributed.worker.memory - WARNING - Ignoring provided memory limit 8.0GiB due to system memory limit of 7.66 GiB
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_target_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.target instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_spill_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.spill instead
  warnings.warn(
/home/zhang/miniconda3/envs/manual-env/lib/python3.10/site-packages/distributed/worker_memory.py:494: FutureWarning: Parameter memory_pause_fraction has been deprecated and will be removed in a future version; please use dask config key distributed.worker.memory.pause 

'http://172.21.212.11:8787/status'

Loading file TABD01110568...
Loading file TABC01142170...

            *********************************************
            * Solar Data Tools Data Onboarding Pipeline *
            *********************************************

            This pipeline runs a series of preprocessing, cleaning, and quality
            control tasks on stand-alone PV power or irradiance time series data.
            After the pipeline is run, the data may be plotted, filtered, or
            further analyzed.

            Authors: Bennet Meyers and Sara Miskovich, SLAC

            (Tip: if you have a mosek [https://www.mosek.com/] license and have it
            installed on your system, try setting solver='MOSEK' for a speedup)

            This material is based upon work supported by the U.S. Department
            of Energy's Office of Energy Efficiency and Renewable Energy (EERE)
            under the Solar Energy Technologies Office Award Number 38529.

            


task list:  71%|████████████████████████▎         | 5/7 [01:11<00:35, 17.71s/it]Traceback (most recent call last):
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/data_handler.py", line 451, in run_pipeline
    self.auto_fix_time_shifts(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/data_handler.py", line 1156, in auto_fix_time_shifts
    self.time_shift_analysis.run(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 76, in run
    hn, rn, tv_metric, jpy, best_ix = self.optimize_w1(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 202, in optimize_w1
    s1, s2 = self.estimate_components(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 241, in estimate_components
    s1, s2 = l2_l1d1_l2d2p365(
  File "/mnt/c/Users/22720/OneDrive/Documents/sol

Fix time shift algorithm failed.
Error message: The solver MOSEK is not installed.




...



total time: 63.65 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              8.83s
Cleaning                   0.86s
Filtering/Summarizing      53.96s
    Data quality           0.35s
    Clear day detect       0.61s
    Clipping detect        11.45s
    Capacity change detect 41.55s

Loading file TAAI01129193...

            *********************************************
            * Solar Data Tools Data Onboarding Pipeline *
            *********************************************

            This pipeline runs a series of preprocessing, cleaning, and quality
            control tasks on stand-alone PV power or irradiance time series data.
            After the pipeline is run, the data may be plotted, filtered, or
            further analyzed.

            Authors: Bennet Meyers and Sara Miskovich, SLAC

            (Tip: if you have a mosek [https://www.mosek.com/] license and have it
            installed on your system, try 

task list:  71%|████████████████████████▎         | 5/7 [01:21<00:40, 20.06s/it]Traceback (most recent call last):
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/data_handler.py", line 451, in run_pipeline
    self.auto_fix_time_shifts(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/data_handler.py", line 1156, in auto_fix_time_shifts
    self.time_shift_analysis.run(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 76, in run
    hn, rn, tv_metric, jpy, best_ix = self.optimize_w1(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 202, in optimize_w1
    s1, s2 = self.estimate_components(
  File "/mnt/c/Users/22720/OneDrive/Documents/solar-data-tools/solardatatools/algorithms/time_shifts.py", line 241, in estimate_components
    s1, s2 = l2_l1d1_l2d2p365(
  File "/mnt/c/Users/22720/OneDrive/Documents/sol

Fix time shift algorithm failed.
Error message: The solver MOSEK is not installed.




task list: 100%|██████████████████████████████████| 7/7 [01:21<00:00, 11.69s/it]




total time: 81.80 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              9.92s
Cleaning                   0.79s
Filtering/Summarizing      71.09s
    Data quality           0.37s
    Clear day detect       0.58s
    Clipping detect        8.95s
    Capacity change detect 61.20s



In [13]:
dask_tool = SDTDask(local_file_data_plug, local_client)

dask_tool.execute(local_file_keys, fix_shifts=True, verbose=True)

## AWS Fargate Client example
use LocalFile dataplug and fargate client

### Instantiate a pvdaq data plug


In [4]:
from sdt_dask.dataplugs.pvdaq_plug import PVDAQPlug

In [5]:
PVDAQPlug?

In [6]:
pvdaq_data_plug = PVDAQPlug()

### Set up a Fargate cluster client


In [7]:
from sdt_dask.clients.aws.fargate import Fargate

In [8]:
# Use should define an environment variable for secret key
PA_NUMBER = os.getenv("project-pa-number")
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')
ENVIRONMENT = {
    'AWS_ACCESS_KEY_ID' : os.getenv('AWS_ACCESS_KEY_ID'),
    'AWS_SECRET_ACCESS_KEY' : os.getenv('AWS_SECRET_ACCESS_KEY')
}

In [9]:
# The Tag, VPC, image, workers, threads per worker and environment need to be user defined and passed to the client class
TAGS = {
    "project-pa-number": PA_NUMBER,
    "project": "pvinsight"
}
VPC = "vpc-ab2ff6d3" # for us-west-2
IMAGE = "smiskov/dask-sdt-sm-2:latest"

WORKERS = 3
THREADS_PER_WORKER = 1

In [ ]:
# print(TAGS, ENVIRONMENT)

In [10]:
fargate_client = Fargate().init_client(image=IMAGE, 
                               tags=TAGS, 
                               vpc=VPC, 
                               region_name=AWS_DEFAULT_REGION,
                               environment=ENVIRONMENT,
                               n_workers=WORKERS,
                               threads_per_worker=THREADS_PER_WORKER
                               )

[i] Initilializing Fargate Cluster ...


C:\Users\Zander\anaconda3\envs\pvi-user\lib\contextlib.py:142: UserWarning: Creating your cluster is taking a surprisingly long time. This is likely due to pending resources on AWS. Hang tight! 
  next(self.gen)


[i] Initilializing Dask Client ...
[>] Dask Dashboard: http://54.245.193.141:8787/status


In [11]:
pvdaq_keys = [(34, 2011), (35, 2015), (51,2012)]
dask_tool = SDTDask(pvdaq_data_plug, fargate_client)

dask_tool.execute(pvdaq_keys, fix_shifts=True, verbose=True)

RuntimeError: Error during deserialization of the task graph. This frequently
occurs if the Scheduler and Client have different environments.
For more information, see
https://docs.dask.org/en/stable/deployment-considerations.html#consistent-software-environments


## Azure Client example